# Forex Recommendation 

In [111]:
import numpy as np 
import pandas as pd 
from datetime import datetime, timedelta
import requests 
import warnings
warnings.filterwarnings("ignore")



In [149]:
# Api key for aplha vantage 
#api_key = '89PMD611NPUTS89W' OLD API KEY 
api_key = '4PNEJW1WVXVPVNB9'

In [150]:
def fetch_forex_data(from_currency, to_currency): # Function to fetch currency pair data for 200 days 
    url = f'https://www.alphavantage.co/query'
    params = {
        'function': 'FX_DAILY',
        'from_symbol': from_currency,
        'to_symbol': to_currency,
        'apikey': api_key,
        'outputsize': 'full'
    }
    response = requests.get(url, params=params)
    data = response.json()
    
    # Extracting the time series data
    time_series = data.get('Time Series FX (Daily)', {})
    
    # Converting to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Adding a column for the currency pair
    df['Currency Pair'] = f'{from_currency}/{to_currency}'
    
    # Filtering data for the last 10 years
    ten_years_ago = datetime.now() - timedelta(days=200)
    df = df[df.index >= ten_years_ago]

    all_data = pd.DataFrame()
    all_data = pd.concat([all_data, df])
    all_data.reset_index(inplace=True)
    all_data.rename(columns={'index': 'Date',"1. open":"Open_price","2. high":"Day_high","3. low":"Day_low","4. close":"Closing_price"}, inplace=True)
    
    #print(all_data.columns)
    # Converting object dtype to numeric
    for i in all_data.select_dtypes('object').columns:
        if i != 'Currency Pair':
            all_data[i] = all_data[i].astype(float)
    
    #print(all_data.columns)
    return all_data





In [151]:
import swing_technical_indicators 
def calculate_technical_indicators(data):
    # Assuming data has the columns: Open, High, Low, Close, Volume
    # Calculate all features using the swing_technical_indicators library
    data['EMA_100'] = swing_technical_indicators.calculate_100ema(data, 'Closing_price', 100)
    data['EMA_200'] = swing_technical_indicators.calculate_200ema(data, 'Closing_price', 200)
    data['EMA_50'] = swing_technical_indicators.calculate_100ema(data, 'Closing_price', 50)
    data['RSI'] = swing_technical_indicators.calculate_rsi(data,'Closing_price',21)
    #print(data.columns)
    macd = swing_technical_indicators.calculate_macd(data, column_name='Closing_price', short_period=12, long_period=26, signal_period=9)
    #print(macd.columns)
    for i in macd.columns:
        data[i] = macd[i]
        #print(i)
        
    bollinger_data = swing_technical_indicators.calculate_bollinger_bands(data['Closing_price'])
    data = data.join(bollinger_data)
    data.drop('Price',axis=1, inplace=True)

    #print(data.columns)
    
    return data

In [154]:
import pandas as pd

def make_recommendation(data):
    # Extract the last week's (7 days) data
    last_week_data = data.tail(7)
    
    # Calculate the latest values for the indicators based on the last week's data
    latest = last_week_data.iloc[-1]  # Latest data point in the last 7 days
    
    # Calculate the trend using the 50 EMA and 200 EMA
    trend = "neutral"  # Default to neutral
    if latest['Closing_price'] > latest['EMA_50'] and latest['Closing_price'] > latest['EMA_200']:
        trend = "bullish"
        print(trend,' ema')
        print(f"The value of EMA 50 : {latest['EMA_50']} and 200 EMA : {latest['EMA_200']} suggest bullish trend")
        
    elif latest['Closing_price'] < latest['EMA_50'] and latest['Closing_price'] < latest['EMA_200']:
        trend = "bearish"
        print(f"The value of EMA 50 : {latest['EMA_50']} and 200 EMA : {latest['EMA_200']} suggest bearish trend")
    #print(trend)
    # Check RSI for the last 7 days
    rsi_signal = "neutral"
    if latest['RSI'] < 30:
        rsi_signal = "bullish"
        #print(f"RSI values: {latest['RSI']} , rsi_signal:{rsi_signal}")
    elif latest['RSI'] > 70:
        rsi_signal = "bearish"
        #print(f"RSI values: {latest['RSI']} , rsi_signal:{rsi_signal}")


    # Check MACD (and MACD signal line) for the last 7 days
    macd_signal = "neutral"
    if latest['MACD'] > 0:
        macd_signal = "bullish"
        print(f"MACD values: {latest['MACD']} , macd: {macd_signal}")
    elif latest['MACD'] < 0:
        macd_signal = "bearish"
        print(f"MACD values: {latest['MACD']} , macd: {macd_signal}")

    # Check Bollinger Bands for the last 7 days
    bb_signal = "neutral"
    if latest['Closing_price'] < latest['Lower Band']:
        bb_signal = "bullish"
        print(f"Bollinger Bands : lower band suggests : {bb_signal}")
    elif latest['Closing_price'] > latest['Upper Band']:
        bb_signal = "bearish"
        print(f"Bollinger Bands : upper band suggests : {bb_signal}")

    # Combine all indicators for decision
      # Default decision is to hold
    target = None
    stop_loss = None
    #print(latest['Closing_price'])
    # Decision making based on the trend, RSI, MACD, and Bollinger Bands
    #print(trend , '1')
    if trend == "bullish" and macd_signal == "bullish":
        #print('yes')
        decision = "BUY"
        target = latest['Closing_price'] * 1.05  # Set target at 5% higher
        stop_loss = latest['Closing_price'] * 0.97  # Set stop loss at 3% lower

        
    elif trend == "bearish"  and macd_signal == "bearish":
        #print('no')
        decision = "SELL"
        target = latest['Closing_price'] * 0.95  # Set target at 5% lower
        stop_loss = latest['Closing_price'] * 1.03  # Set stop loss at 3% higher
    else :
        decision = "WAIT"

    #print(trend)
    #print(decision, target, stop_loss)
    return decision, target, stop_loss


In [155]:
def trading_pipeline(symbol_from, symbol_to, api_key, days=200):
    # Fetch forex data
    data = fetch_forex_data(symbol_from, symbol_to)
    
    # Calculate technical indicators
    data = calculate_technical_indicators(data)
    
    # Make recommendation
    decision, target, stop_loss = make_recommendation(data)
    print('Currency pair : ',symbol_from, symbol_to)
    # Print recommendation and details
    print(f"Recommendation: {decision}")
    if decision != "WAIT":
        print(f"Target Price: {target:.4f}")
        print(f"Stop Loss: {stop_loss:.4f}")
        print(f"- Target/Stop Loss set at 5% and 3% based on the current price.")
        print("Recommendation Based on the Following Justifications:")
    
        
    
    elif decision == "WAIT":
        print(f"- Hold for 1 week to 1 month.")
    else:
        print(f"- Set a stop loss at {stop_loss:.4f} and target price at {target:.4f}.")
    print(f"- Trend: {decision} trend confirmed by indicators.")

# Example usage

symbol_from = 'USD'
symbol_to = 'INR'

# Run the pipeline
trading_pipeline(symbol_from, symbol_to, api_key, days=200)

bullish  ema
The value of EMA 50 : 84.19416309732763 and 200 EMA : 83.81133173034327 suggest bullish trend
MACD values: 0.09468282668794359 , macd: bullish
Currency pair :  USD INR
Recommendation: BUY
Target Price: 88.7880
Stop Loss: 82.0232
- Target/Stop Loss set at 5% and 3% based on the current price.
Recommendation Based on the Following Justifications:
- Trend: BUY trend confirmed by indicators.
